# Preprocesamiento

En esta etapa generamos las imágenes que utilizará el modelo para el entrenamiento y la predicción. 

Primero se generan las imágenes de n-bandas. Es decir las imágenes que no solo poseen los canales RGB sino además información de otras bandas como el infrarrojo cercano (NIR near-infra-red),  o mismo, una combinación de bandas. En esta etapa consideramos toda la información que podría ser sensible a la detección del objeto de interés.

Luego, utilizando las imágenes generadas en el paso anterior y la información vectorial de la ubicación de los basurales, generamos el dataset utilizado luego por el modelo de ML. Este dataset se genera con el formato necesario para el modelo.



## Preprocesamiento para las imagenes de entrenamiento


In [1]:
import glob
import os
from tqdm import tqdm

### Download images

In [ ]:
# !gsutil -m cp -r gs://dym-temp/basurales/train_6D_/* .

### Setup


In [2]:
PATHS = ['/home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/*.tif'#,
         #'/home/ro/basurales/data/img/img_predict/2021-02-01_2021-02-28/*.tif',
         ]


### Compute  NDSW index


Generamos la banda Normalized Difference ShortWave (NDSW) como la diferencia normalizada entre las dos bandas del infrarrojo de onda corta, es decir NDSW = (SWIR12-SWIR11)/(SWIR12 + SWIR11 ) . Esta diferencia de bandas mostró ser sensible a la emisión de metano en estudios previos.  Dicho componente puede ser un indicador de la presencia de un basural con residuos biológicos.


In [3]:

# para cada imagen en los grupos de entrenamiento prediccion y control, generamos la banda NDSW
for path in PATHS:
   
    images = glob.glob(path) #ruta a cada imagen 
    
    for image in tqdm(images):
        
        #skip resulting files 
        if "_diffSW" in image: continue
        if "_ori_tmp" in image: continue
        if "rgb-nir-sw-diffSW"  in image: continue
            
        fname, ext = os.path.splitext(image)
        
        #--------------------------------
        #  Generacion de la banda NDSW  
        #--------------------------------
        
        ndsw_file = str(fname+'_diffSW.tif')
        #usamos gdal_calc para calcular la nueva banda usando la banda 5 y 6 de cada imagen
        !gdal_calc.py -A $image --A_band=5 -B $image --B_band=6 --calc="(A-B)/(A+B)" --outfile=$ndsw_file 
        
        #------------------------------------------
        #  Generacion de la bandas RGB - NIR -SWIR  
        #------------------------------------------
        
        img_5bands = str(fname+'_ori_tmp.tif')
        #usamos gdal_translate quedarnos con las 5 primeras bandas de cada imagen
        !gdal_translate -b 1 -b 2 -b 3 -b 4 -b 5 $image $img_5bands

        #----------------------------------------------
        #  Generacion de imagen RGB - NIR -SWIR -NDSW 
        #----------------------------------------------        
        
        rgb_nir_swir_diffSW_fname = str(fname+'_rgb-nir-sw-diffSW.tif')
        #usamos otbcli_ConcatenateImages para concatenar las imagenes generadas en el paso anterior quedarnos con las 5 primeras bandas de cada imagen
        !bash -c "source /opt/OTB/otbenv.profile; otbcli_ConcatenateImages -il  $img_5bands $ndsw_file -out $rgb_nir_swir_diffSW_fname"


  0%|          | 0/42 [00:00<?, ?it/s]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2021-07-07 14:03:00 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2021-07-07 14:03:00 (INFO) ConcatenateImages: GDAL maximum cache size is 750 MB
2021-07-07 14:03:00 (INFO) ConcatenateImages: OTB will use at most 4 threads
2021-07-07 14:03:00 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2021-07-07 14:03:00 (INFO): File /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000067840-0000013568_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000067840-0000013568_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (23s)


  2%|▏         | 1/42 [02:02<1:23:34, 122.30s/it]

0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 9.. 9.. 9.. 9.. 9.. 9.. 10.. 10.. 10.. 10.. 10.. 10.. 11.. 11.. 11.. 11.. 11.. 12.. 12.. 12.. 12.. 12.. 12.. 13.. 13.. 13.. 13.. 13.. 13.. 14.. 14.. 14.. 14.. 14.. 14.. 15.. 15.. 15.. 15.. 15.. 15.. 16.. 16.. 16.. 16.. 16.. 16.. 17.. 17.. 17.. 17.. 17.. 18.. 18.. 18.. 18.. 18.. 18.. 19.. 19.. 19.. 19.. 19.. 19.. 20.. 20.. 20.. 20.. 20.. 20.. 21.. 21.. 21.. 21.. 21.. 21.. 22.. 22.. 22.. 22.. 22.. 22.. 23.. 23.. 23.. 23.. 23.. 24.. 24.. 24.. 24.. 24.. 24.. 25.. 25.. 25.. 25.. 25.. 25.. 26.. 26.. 26.. 26.. 26.. 26.. 27.. 27.. 27.. 27.. 27.. 27.. 28.. 28.. 28.. 28.. 28.. 28.. 29.. 29.. 29.. 29.. 29.. 30.. 30.. 30.. 30.. 30.. 30.. 31.. 31.. 31.. 31.. 31.. 31.. 32.. 32.. 32.. 32.. 32.. 32.. 33.. 33.. 33.. 33.. 33.. 33.. 34.. 34.. 34.. 34.. 34.. 34.. 35.. 35.. 35.. 35.. 35.. 36.. 36..

...

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2021-07-07 15:07:27 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2021-07-07 15:07:27 (INFO) ConcatenateImages: GDAL maximum cache size is 750 MB
2021-07-07 15:07:27 (INFO) ConcatenateImages: OTB will use at most 4 threads
2021-07-07 15:07:27 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2021-07-07 15:07:27 (INFO): File /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000000000-0000000000_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000000000-0000000000_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (24s)


 98%|█████████▊| 41/42 [1:06:30<02:05, 125.23s/it]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2021-07-07 15:09:46 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2021-07-07 15:09:46 (INFO) ConcatenateImages: GDAL maximum cache size is 750 MB
2021-07-07 15:09:46 (INFO) ConcatenateImages: OTB will use at most 4 threads
2021-07-07 15:09:46 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2021-07-07 15:09:46 (INFO): File /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000013568-0000000000_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing /home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/0000013568-0000000000_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (25s)


100%|██████████| 42/42 [1:08:51<00:00, 98.36s/it] 


## Generación del dataset de entrenamiento

El modelo de ML utiliza para entrenar un dataset compuesto por imágenes y máscaras binarias que delimitan el objeto de interés. Estas se generan vía la herramienta **satproc_extract_chips**. Para el caso del dataset de entrenamiento se generan imágenes y máscaras (utilizando las anotaciones de verdad de campo) y para el de predicción solo imágenes.

Para generar las imágenes de entrenamiento:

In [5]:
#!pip install pysatproc

In [6]:
path_to_files = "/home/ro/basurales/data/img/img_train/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif"
output_folder = "/home/ro/basurales/data/data_train/6D/100_30/"

vector_file = "/home/ro/basurales/data/shp/gt/gt_basurales_4326.geojson"

size = 100
step_size = 30

In [7]:
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file \
    --labels $vector_file \
    --label-property 'class' \
    --classes 'B' \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100

  0%|                                                    | 0/42 [00:00<?, ?it/s]
0it [00:00, ?it/s]
 10%|████▏                                       | 4/42 [00:30<04:59,  7.88s/it]
0it [00:00, ?it/s]
 12%|█████▏                                      | 5/42 [00:36<04:25,  7.16s/it]
0it [00:00, ?it/s]
 44%|██████████████████▋                       | 67/151 [00:01<00:01, 53.17it/s]


 24%|██████████▏                                | 10/42 [01:08<03:48,  7.15s/it]
0it [00:00, ?it/s]
 26%|███████████▎                               | 11/42 [01:13<03:29,  6.75s/it]
0it [00:00, ?it/s]
 33%|██████████████▎                            | 14/42 [01:28<02:44,  5.88s/it]
0it [00:00, ?it/s]
  5%|██▏                                         | 3/59 [00:00<00:01, 28.57it/s]


 55%|███████████████████████▌                   | 23/42 [02:23<02:02,  6.47s/it]
0it [00:00, ?it/s]
 69%|█████████████████████████████▋             | 29/42 [02:53<01:13,  5.68s/it]
0it [00:00, ?it/s]
 71%|██████████████████████████████▋            | 30/42 [02:54<00:51,  4.25s/it]
0it [00:00, ?it/s]
 74%|███████████████████████████████▋           | 31/42 [02:58<00:46,  4.24s/it]
0it [00:00, ?it/s]
 76%|████████████████████████████████▊          | 32/42 [03:02<00:42,  4.21s/it]
0it [00:00, ?it/s]
 79%|█████████████████████████████████▊         | 33/42 [03:07<00:38,  4.24s/it]
0it [00:00, ?it/s]
 81%|██████████████████████████████████▊        | 34/42 [03:11<00:33,  4.21s/it]
0it [00:00, ?it/s]
 83%|███████████████████████████████████▊       | 35/42 [03:15<00:29,  4.20s/it]
0it [00:00, ?it/s]
 86%|████████████████████████████████████▊      | 36/42 [03:16<00:18,  3.16s/it]
0it [00:00, ?it/s]
 88%|█████████████████████████████████████▉     | 37/42 [03:16<00:11,  2.38s/it]
0it [00:00, ?it/s]


###### Los argumentos:

* **El primer argumento** es la ruta a las imágenes 

* **o** es la ruta de destino 

Recomendamos que dicha ruta sea descriptiva, por ejemplo “data_train/600_600/ ” describe : Data_train → datos usados para entrenar; 400_400 → <tamaño de la imagen >_ <tamaño del step-size> (las imágenes son cuadradas)

* **size** tamaño de las imágenes resultantes (las imágenes son cuadradas) 
* **step-size** paso del proceso. Si *step-size* es igual que el *size* entonces no hay overlap en las imágenes. 

En ocaciones es útil para el entrenamiento generar los chips con un overlap de este modo tenemos más datos para entrenar. Pero en la predicción valor debe ser igual al tamaño que la imagen 

* **crs** epsg: le asigna un epsg a la imagen 

* **label-property** nombre del campo donde se define cada categoría (solo se usa para el entrenamiento) 

* **classes** nombres de las clases (como aparecen en el geojson), separados por espacios

* **aoi** ruta al archivo vectorial donde están definidas las localidades. Al definir una region de interés solo se procesan las imágenes que interceptan esas localidades.

* **rescale** lleva los valores de las bandas a 0-255

Este comando va a generar dos carpetas en la ruta de destino : “images” y “masks”. Los archivos de la primera van a ser de tipo Tiff de 3 bandas (rgb) y los de la segunda van a ser, también, de tipo Tiff pero de N bandas donde N representa el número de clases, en este caso sólo una. Y donde cada una de las bandas es una máscara binaria


## Generación del dataset de predicción y control


De modo similar repetimos el procedimiento. En este caso el unico archivo vectorial delimita las zonas de interés para la predicción. 

### dataset de predicción

In [ ]:
path_to_files = "/home/ro/basurales/data/img/img_predict/*/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif"
output_folder = "/home/ro/basurales/data/data_predict/6D/100_30/"

vector_file_aoi = "/home/ro/basurales/data/shp/aoi-predict.geojson"

size = 100


!satproc_extract_chips \
    $path_to_files \
    -o $output_folder \
    --size $size \
    --step-size $size \
    --aoi $vector_file_aoi \
    --rescale \
    --rescale-mode custom_rgb --lower-cut 1 --upper-cut 100